# SQL Databases on hadoop

Relationele Database Management Systemen zijn traditioneel verticaal schaalbaar.
Dit conflicteert met Big Data toepassingen waar we de voorkeur geven aan horizontale schaalbaarheid om onder andere de kosten te verlagen, de beschikbare rekenkracht of performantie en fout-tolerantie te verbeteren.
Hierdoor kunnen de reeds gekende RDBMS niet gebruikt worden.

De volgende populaire applicaties kunnen gebruikt worden om SQL te gebruiken op hadoop:
* Apache Hive
* Cloudera Impala
* Presto
* Shark

Hierna gaan we een kort overzicht geven van al deze applicaties en daarna focussen we op een toepassingen van Apache Hive.

## Apache Hive

Deze applicatie is ontwikkeld door Facebook als **datawarehouse framework** voor interne toepassingen en is snel zeer populair geworden om queries uit te voeren op Hadoop.
Het is belangrijk om in gedachten te houden dat hoewel Hive een **SQL-like querying omgeving** aanbiedt, dat er in de achtergrond een **MapReduce methodologie** gebruikt wordt om de database te bevragen.
Dit wil zeggen dat de **queries gecompileerd moeten worden naar MapReduce toepassingen**.
Hive ondersteund ook gebruikers-gedefineerde functies en laat toe om gecomprimeerde data te verwerken.
Momenteel wordt Hive verder verbeterd en uitgebreid door HortonWorks (Cloudera) dat een nieuwe backend aan het uitwerken is (Tez Project) om de responstijd van Hive te verbeteren.

Voordelen:
* Op bijna alle Hadoop installaties standaard geinstalleerd
* Goede tool om te proberen door minimale start-investering (gratis)

Nadelen:
* Niet meest snelle manier door overhead van MapReduce (batch processing)
* Enkel 4 file-formats ondersteund:
 * Text, SequenceFile, ORC, RCFile

## Cloudera Impala

Maakt het mogelijk om interactieve SQL queries uit te voeren op HDFS en HBase. 
Impala voert **queries uit in real time** en verbeterd daardoor de performantie  door geen batch processing te gebruiken.
Daarnaast wordt ook het **gebruik van verscheidene SQL-based bedrijfsanalyse tools mogelijk** gemaakt.
Deze applicatie is een open source applicatie ontwikkeld door Cloudera.

Voordelen:
* Sneller dan Hive
* Ondersteund cloud based architecture door Amazon's Elastic MapReduce
* Is compatibel met ANSI SQL (standaard SQL standaard)
* Integratie met business intelligence tools mogelijk

Nadelen:
* Moeilijker op te zetten
* Volledige kracht maar beschikbaar bij gebruik van Parquet file format
* Geen support voor YARN
* Vereist installatie van daemons op elke node

## Presto

Een tweede applicatie ontwikkeld door Facebook.
Ook deze applicatie is open source.
Deze applicatie is geschreven in Java en heeft een groot aantal kenmerken gemeen met Impala, bijvoorbeeld:
* Een interactieve ervaring
* Moeiljk om op te zetten (installatie op de verscheidene nodes)
* Vereist een specifiek file format voor data opslag (RCFile)

Daarnaast biedt Presto wel compatibiliteit met de Hive meta-store en laat Presto toe om data van verscheidene bronnen te combineren.
Het grootste verschil met Impala is dat Presto niet ondersteund wordt door veel leverancies van cloud-toepassingen, ook al maken reeds een aantal grote bedrijven er gebruik van (bijvoorbeeld AirBnb en Dropbox).

## Shark

Deze applicatie is ontstaan om een alternatief te bieden voor Hive met MapReduce.
Het doel was om alle functionaliteiten van Hive te behouden maar de performantie te verbeteren.
Deze tool is geschreven in Scala door UC Berkeley en zoals de naam doet vermoeden maakt het gebruik van **Spark**.
Tot op een zeker punt kan Shark de performantie van Hive verbeteren maar de **schaalbaarheid van de tool** is niet zo goed als Hive.
Dit komt omdat het **gebouwd is boven op Hive** waardoor het de complexe codebase van Hive heeft overgeerfd heeft.
Het onderhoud en aanpassen van deze codebase zonder in te boeten op performantie is echter niet eenvoudig.

## Spark SQL

Dit onderdeel van Spark biedt de mogelijkheid aan om Spark Queries uit te voeren op ingeladen Dataframes.
Omdat dit gebruik maakt van Spark biedt het veel voordelen en is de performantie beter dan tools die gebruik maken van MapReduce.
Het grootste nadeel is echter dat deze data niet standaard opgeslagen wordt op een harde schrijf.
Het is vrij eenvoudig om deze dataframes/tabellen op te slaan als bijvoorbeeld csv maar de relaties tussen kolommen van verschillende tabellen kan niet opgeslaan worden en vereist extra manueel werk om bij te houden.

## Voorbeeld toepassing: Hive

Om te beginnen moet Hive geinstalleerd worden.
Hiervoor kan je de stappen volgen op [deze pagina](https://phoenixnap.com/kb/install-hive-on-ubuntu).
Neem hiervoor de laatste versie (en de standaard hive, niet storage) en kies ook voor de reeds gecompileerde files(bin), niet de download met source files.
De meest recente versie op het moment van het schrijven van deze file en de versie die ik geinstalleerd heb staan is  hive 3.1.2 met deze download file: apache-hive-3.1.2-bin.tar.gz.
Let bij het installeren dat je de juiste paden invult naar de gedownloade folder (ik heb een hive folder aangemaakt in mijn home directory).

**Let op: Het laatste deel van stap 5 (aanpassen van de hive.metastore.warehouse.dir property) moet niet uitgevoerd worden. Wacht met het uitvoeren van stap 6 tot onderstaande problemen zijn opgelost**

**Probleem 1: gebruik van oudere versies door Hive**

Ten eerste maakt Hive gebruik van een oudere versie van guave.
Hoe je dit kan oplossen staat na de beschrijving van stap 6.

Ook is Hive gecompileerd voor Java 8 en standaard is Java 11 geinstalleerd.
Installeer nu Java 8 met het volgende commando

    sudo apt install openjdk-8-jdk
    
Gebruik dan het onderstaande commando om de default versie op het hele systeem aan te passen (kies hierbij versie 8)

    sudo update-alternatives --config java
    
Ten slotte open je in de home het .bashrc file en pas je de JAVA_HOME variabele aan naar versie 8.
Vergeet niet deze aanpassingen door te voeren door middel van het volgende commando (indien je terminal in je home directory staat).

    source .bashrc
    
Een laatste aanpassing voor java is dat de link in ~/hadoop/etc/hadoop/hadoop-env.sh nog niet aangepast is. Pas het pad naar de java versie 8 aan helemaal onderaan deze file.
Doe dit ook voor de spark-env.sh. 
Deze file bestaat nog niet maar een template kan je vinden in hadoop/spark.../conf.
Kopieer deze template en hernoem ze naar spark-env.sh.
Restart nu het hdfs zodat de aanpassingen meegenomen worden.

**Probleem 2: illegal character**

Een ander probleem is dat de nieuwe guave versie die we gebruiken een bepaald character niet herkend in de hive-site.xml. 
Dit karakter (4 symbolen begint met en &) kan je vinden op rij 3215 van hive-site.xml.
Aangezien dit karakter in een eenvoudige beschrijving staat kan het verwijderd worden zonder problemen.
Verwijder dit karakter en sla je aanpassingen op. 

**Probleem 3: instellen user name**

Hive moet ook nog weten als welke user het moet inloggen op het hdfs.
Plaats hiervoor het volgende in ~/hive/conf/hive-site.xml
    
    <property>
        <name>system:java.io.tmpdir</name>
        <value>/tmp/hive/java</value>
    </property>
    <property>
        <name>system:user.name</name>
        <value>${user.name}</value>
    </property>    
    
**Ga nu verder met de installatie door het uitvoeren van stap 6**

### Hive via commandline

We kunnen nu hive gebruiken via commandline door in de terminal het volgende commando uit te voeren

    hive
    
Dit opent een sql-like console waarin we allerhande queries kunnen schrijven.
Een uitgebreide beschrijving van alle mogelijke queries vind je [hier](https://cwiki.apache.org/confluence/display/hive/languagemanual).
Bijvoorbeeld kunnen we de databases bekijken met

    show databases;
    
Maak ook een database aan waarin we we gaan werken deze les.
Schrijf hieronder de commando's die je nodig hebt om dit te doen.

In [ ]:
!create database odisee;
!use odisee;

Download nu deze zip files over [werknemers](https://github.com/RobinDong/hive-examples/blob/master/employee/employees.csv.gz) en [salarissen ](https://github.com/RobinDong/hive-examples/blob/master/employee/salaries.csv.gz).
Unzip daarna deze files en upload ze naar het hdfs.
Schrijf hieronder de nodige code om deze files in het hdfs op te slaan.

In [1]:
import pydoop.hdfs as hdfs

localFS = hdfs.hdfs(host='')
client = hdfs.hdfs(host='localhost', port=9000)

if not client.exists('/user/bigdata/10_SQL'):
    client.create_directory('/user/bigdata/10_SQL')

# do some cleaning in case anything is present
for f in client.list_directory("."):
    client.delete(f["name"], True)
        
# upload input.txt
localFS.copy("/home/bigdata/Downloads/employees.csv", client, "10_SQL/employees/employees.csv")
localFS.copy("/home/bigdata/Downloads/salaries.csv", client, "10_SQL/salaries/salaries.csv")

2022-02-24 15:29:09,657 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


0

Deze files bevatten de data die we straks gaan inlezen in het Hive Datawarehouse. 
Hiervoor moeten we echter eerst de tabellen aanmaken waarin we deze data gaan opslaan.
Dit kan door middel van de volgende HQL commando's uit te voeren in de hive HQL.

Na het maken kunnen we de data inlezen door middel van het load data commando.

In [ ]:
create external table employee (
    employee_id INT,
    birthday DATE,
    first_name STRING,
    family_name STRING,
    gender CHAR(1),
    work_day DATE)
row format serde 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
stored as textfile;

LOAD DATA INPATH '/user/bigdata/10_SQL/employees' overwrite into table employee;

create external table salary (
    employee_id INT,
    salary INT,
    start_date DATE,
    end_date DATE)
row format serde 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
stored as textfile

LOAD DATA INPATH '/user/bigdata/10_SQL/salaries' overwrite into table salary;

Bestudeer nu opnieuw de [HQL language](https://cwiki.apache.org/confluence/display/Hive/LanguageManual) en stel de queries op die de volgende zaken opzoeken:
* De 10 oudste werknemers
* Het aantal werknemers dat gestart is in 1990
* De voor en familienaam en gemiddelde salaris van de 10 werknemers met het hoogste gemiddelde salaris. (Tip: gebruik order by ipv sort by om globale orde te bepalen in reducer)
* Is er een gender wage gap aanwezig in dit bedrijf? Bepaal hiervoor per geslacht het gemiddelde salaris aan de hand van een group by.

In [ ]:
select * from employee order by birthday asc limit 10;

In [ ]:
select count(*) from employee where work_day >= '1990-01-01' and work_day <= '1990-01-31';

In [ ]:
select e.first_name, e.family_name, avg(s.salary) as avg_salary from
    employee as e join salary as s on (e.employee_id == s.employee_id)
        group by e.first_name, e.family_name order by avg_salary limit 10;

In [ ]:
SELECT e.gender, AVG(s.salary) AS avg_salary
    FROM employee AS e
          JOIN salary AS s
            ON (e.employee_id == s.employee_id)
GROUP BY e.gender;

Buiten rechtstreeks te werken met de hive interface om rechtstreeks queries uit te voeren kunnen we dit ook met spark doen.
Hieronder staan de bovenstaande commando's omgezet naar queries met spark om het binnen een spark applicatie uit te voeren.

In [ ]:
from os.path import abspath

from pyspark.sql import SparkSession
from pyspark.sql import Row

# warehouse_location points to the default location for managed databases and tables
warehouse_location = abspath('spark-warehouse')

spark = SparkSession.builder.appName("Spark_Hive_Les").config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport().getOrCreate()

spark.sql("""create external table employee (employee_id INT, birthday DATE, first_name STRING, family_name STRING,
          gender CHAR(1), work_day DATE) row format serde 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
          stored as textfile USING hive;""")

spark.sql("LOAD DATA INPATH '/user/bigdata/10_SQL/employees' overwrite into table employee;")

spark.sql("select * from employee order by birthday asc limit 10;").show()